Import

In [1]:
import os
import openai
import sys

sys.path.append("../..")

from dotenv import load_dotenv, find_dotenv

env_path = find_dotenv()
print(f"Path of .env : {env_path}")


_ = load_dotenv(find_dotenv())  # read local .env file

openai.api_key = os.environ["OPENAI_API_KEY"]

Chemin du fichier .env trouvé : /home/guillaume/Code/forestbot/.env


Document

In [2]:
from langchain_community.document_loaders import PyPDFLoader

# def load_pdf_with_metadata(file_path):
#     loader = PyPDFLoader(file_path)
#     documents = loader.load()
#     # Assign filename to each document's metadata
#     for document in documents:
#         document.metadata['filename'] = file_path
#     return documents

# docs1 = load_pdf_with_metadata("../pdf/EdAP 2020_EN.pdf")
# docs2 = load_pdf_with_metadata("../pdf/SOF book-web-rev3d-hires.pdf")

# documents = docs1 + docs2

loaders = [
    PyPDFLoader("../pdf/EdAP 2020_EN.pdf"),
    PyPDFLoader("../pdf/SOF book-web-rev3d-hires.pdf"),
]

documents = []
for loader in loaders:
    documents.extend(loader.load())

for document in documents:
    document.metadata["filename"] = document.metadata["source"]

# for document in documents:
#    document.metadata['filename'] = file_path

# Print metadata for checking
# for document in documents:
#    print(f"Document from: {document.metadata['filename']}")

In [12]:
# Print metadata for checking
# for document in documents:
#    print(f"Document from: {document.metadata['filename']}")
# print(documents[0])
# print(type(documents))
# print(documents[0].keys())
# print(dir(documents[0]))
# Afficher le contenu de la page du document
print("Contenu de la page :", documents[0].page_content)

# Afficher les métadonnées du document
print("Métadonnées :", documents[0].metadata)

Contenu de la page : STATE OF PROTECTED AREAS IN CENTRAL AFRICA 2020STATE OF 
PROTECTED
AREAS
IN CENTRAL 
AFRICA
2020

Métadonnées : {'source': '../pdf/EdAP 2020_EN.pdf', 'page': 0, 'filename': '../pdf/EdAP 2020_EN.pdf'}


Generate data set

In [10]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import logging

# Configuration of logging
logging.basicConfig()
logger = logging.getLogger("tenacity")

# Define models
generator_llm = ChatOpenAI(model="gpt-3.5-turbo")
critic_llm = ChatOpenAI(model="gpt-4")
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(generator_llm, critic_llm, embeddings)

testset = generator.generate_with_langchain_docs(
    documents[0:49],
    test_size=10,
    distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25},
)

embedding nodes:   0%|          | 0/114 [00:00<?, ?it/s]

Generating:   0%|          | 0/10 [00:00<?, ?it/s]

Save dataset

In [28]:
df_testset = testset.to_pandas()
df_testset.to_csv(
    "evaluation_dataset.csv", index=False
)  # `index=False` to not include label in the file

In [29]:
print(df_testset.columns)
print(df_testset.head)

Index(['question', 'contexts', 'ground_truth', 'evolution_type', 'metadata',
       'episode_done'],
      dtype='object')
<bound method NDFrame.head of                                             question  \
0  How do the Aichi Targets promote coordinated a...   
1  What is the purpose of the Observatory of Biod...   
2  How do protected areas in Central African coun...   
3  How does habitat destruction pose a threat to ...   
4  What were the broad objectives for the overhau...   
5  What actions are needed for Aichi Targets to a...   
6  How do certified forestry concessions help eco...   
7  How do forest concessions in the DRC help biod...   
8  What is the main goal of SOPA reports and how ...   
9  How do Central African protected areas support...   

                                            contexts  \
0  [ are promoted through the Aichi Targets. They...   
1  [MINFOF: Ministère des Forêts et de la Faune (...   
2  [40\nVarious COMIFAC directives, notably those...   
3  [32